<a href="https://colab.research.google.com/github/Chitrak07/Kaggle/blob/main/godaddy_microbusiness_density_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shubhamkumarvaish","key":"e9b84c4d3ac13462bad15f7f3412e375"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c godaddy-microbusiness-density-forecasting

  0% 0.00/1.74M [00:00<?, ?B/s]
100% 1.74M/1.74M [00:00<00:00, 163MB/s]


In [ ]:
!unzip /content/godaddy-microbusiness-density-forecasting.zip

Archive:  /content/godaddy-microbusiness-density-forecasting.zip
replace census_starter.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: census_starter.csv      
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [ ]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
path = '/content/'

In [ ]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
train["first_day_of_month"] = pd.to_datetime(train["first_day_of_month"])
test["first_day_of_month"] = pd.to_datetime(test["first_day_of_month"])

In [ ]:
train.describe()

,cfips,microbusiness_density,active
count,122265.000000,122265.000000,1.222650e+05
mean,30376.037640,3.817671,6.442858e+03
std,15143.508721,4.991087,3.304001e+04
min,1001.000000,0.000000,0.000000e+00
25%,18177.000000,1.639344,1.450000e+02
50%,29173.000000,2.586543,4.880000e+02
75%,45077.000000,4.519231,2.124000e+03
max,56045.000000,284.340030,1.167744e+06


In [ ]:
print(train.columns.tolist())

['row_id', 'cfips', 'county', 'state', 'first_day_of_month', 'microbusiness_density', 'active']


In [ ]:
train.head

<bound method NDFrame.head of                   row_id  cfips          county    state first_day_of_month  \
0        1001_2019-08-01   1001  Autauga County  Alabama         2019-08-01   
1        1001_2019-09-01   1001  Autauga County  Alabama         2019-09-01   
2        1001_2019-10-01   1001  Autauga County  Alabama         2019-10-01   
3        1001_2019-11-01   1001  Autauga County  Alabama         2019-11-01   
4        1001_2019-12-01   1001  Autauga County  Alabama         2019-12-01   
...                  ...    ...             ...      ...                ...   
122260  56045_2022-06-01  56045   Weston County  Wyoming         2022-06-01   
122261  56045_2022-07-01  56045   Weston County  Wyoming         2022-07-01   
122262  56045_2022-08-01  56045   Weston County  Wyoming         2022-08-01   
122263  56045_2022-09-01  56045   Weston County  Wyoming         2022-09-01   
122264  56045_2022-10-01  56045   Weston County  Wyoming         2022-10-01   

        microbusiness

In [ ]:
percent_missing = train.isnull().sum() * 100 / len(train)
print(percent_missing)

row_id                   0.0
cfips                    0.0
county                   0.0
state                    0.0
first_day_of_month       0.0
microbusiness_density    0.0
active                   0.0
dtype: float64


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   row_id                 122265 non-null  object        
 1   cfips                  122265 non-null  int64         
 2   county                 122265 non-null  object        
 3   state                  122265 non-null  object        
 4   first_day_of_month     122265 non-null  datetime64[ns]
 5   microbusiness_density  122265 non-null  float64       
 6   active                 122265 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 6.5+ MB


In [ ]:
import xgboost as xgb

In [ ]:
THRESHOLD = 8
ACTIVE_THRESHOLD = 2000
TRAIN_SZ = len(train)//3135  # 39
TEST_SZ = len(test)//3135    # 8

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
ids = train.cfips.unique() # Unique identifiers for regions

x_train = np.arange(TRAIN_SZ).reshape((-1,1))
x_test = np.arange(TRAIN_SZ-1,TRAIN_SZ+TEST_SZ).reshape((-1,1))

linear_preds = np.zeros(len(ids))
last_preds = np.zeros(len(ids))
seasonal_preds = np.zeros(len(ids))

sn_trend = 0
lin_trend = 0
ct=0
for i, c in enumerate(ids):
    df = train.loc[train.cfips == c]
    
    last = df.microbusiness_density.values[-1]
    active = df.active.values[-1]
    
    last_preds[i] = last
    
    # Seasonal forecast
    WIDTH1 = 5; WIDTH2 = 7; WIDTH3 = 7

   # Recent trend
    x0a = df.microbusiness_density.values[-1-WIDTH1:-1]
    x0 = np.median(x0a)
    # 1 year old trend
    x1 = df.microbusiness_density.values[-12-1+1]
    x2a = df.microbusiness_density.values[-12-1-WIDTH2+1:-12-1+1]
    x2 = np.median(x2a)
  
    x3 = df.microbusiness_density.values[-24-1+1]
    x4a = df.microbusiness_density.values[-24-1-WIDTH3+1:-24-1+1]
    x4 = np.median(x4a)


    p = last 
    if active >= ACTIVE_THRESHOLD:         
        if (x1 > x2) and (x3 > x4) and (last > x0):
            p *= 1.004
        elif (x1 < x2) and (x3 < x4) and (last < x0):
            p *= 0.996
    seasonal_preds[i] = p
    
    # Linear regression
   # model = LinearRegression()
   # model.fit(x_train, df.microbusiness_density)
  #p = model.predict(x_train)
   # err = p - df.microbusiness_density.values
   # rng = df.microbusiness_density.max() - df.microbusiness_density.min()
   
    #RandomForestRegressor  xgb
    
    model = LinearRegression()
    model.fit(x_train, df.microbusiness_density)
    p = model.predict(x_train)
    err = p - df.microbusiness_density.values
    rng = df.microbusiness_density.max() - df.microbusiness_density.min()

    # Determine if the time series is linear or not
    s = 0
    for k in range(TRAIN_SZ):
        e = np.abs(err[k])
        r = e / (rng/2)
        s += r
    if (s > THRESHOLD) or (active < ACTIVE_THRESHOLD):
        # If the time series is not linear or activity is below the threshold
        # values use last value as prediction
        linear_preds[i] = last
    else:
        # Otherwise, use linear regression
        p2 = model.predict(x_test)
        shift = last - p2[0]
        linear_preds[i] = p2[1] + shift
        lin_trend += 1
        
# Final predictions using a combination of linear,
# seasonal and last value forecasts
final_preds = (linear_preds + last_preds + seasonal_preds) / 3

In [ ]:
target = pd.DataFrame(data={"microbusiness_density":final_preds}, index=ids)

In [ ]:
test = test.join(target, on="cfips")[["row_id", "microbusiness_density"]]

In [ ]:
test.to_csv("submissionl2.csv", index=False)